In [20]:
from os import path, makedirs
from pickle import dump, load
from re import compile, sub
from requests import get, exceptions
from itertools import chain
from nltk.tokenize import sent_tokenize
from json import JSONDecodeError
from pandas import DataFrame, concat

Регулярки

In [2]:
def cut(data):
    r = compile(r'<.*?>|>>\d*|\(OP\)|&#(\d*);|&quot;|&gt;|(http|https):.*')
    return r.sub('', punctuate_word(punctuate_sent((data))))

def punctuate_sent(data):
    r = compile(r'([a-zA-Zа-яА-Я])([.!\?])')
    return r.sub(r'\1. ', data)

def punctuate_word(data):
    r = compile(r'([a-zA-Zа-яА-Я])([,])')
    return r.sub(r'\1, ', data)

Получение списка тредов доски

In [38]:
BOARD = 'pr'

In [39]:
DVACH = 'https://2ch.hk/'

try:
    dvach_page = get(DVACH + BOARD + '/catalog.json').json()
    threads = [i['num'] for i in dvach_page['threads']]
except JSONDecodeError:
    print('No board with such name.')

Формирование корпуса сообщений доски

In [40]:
comments = []

for every_thread in threads:
    try:
        thread = get(DVACH +  BOARD + '/res/' + every_thread + '.json', timeout=5).json()
        [comments.append(sent_tokenize(cut(i['comment']))) for i in thread['threads'][0]['posts'] if len(cut(i['comment'])) > 2]
    except:
        pass
    
comments = list(chain.from_iterable(comments))

Получение датафрейма

In [41]:
df = DataFrame()
df['comment'] = comments
df.index.names = ['comment_id']

Загрузка старого корпуса и мердж

In [42]:
try:
    file_path = path.join(subdir, BOARD + '.csv')
    old_df = DataFrame.from_csv(file_path)
    df = concat((df, old_df)).drop_duplicates()
except FileNotFoundError:
    print('No pre-serialized data!')

No pre-serialized data!


Сериализация корпуса

In [43]:
subdir = 'pickle'

if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, BOARD + '.csv')

df.to_csv(file_path)